# 사용자 정의 Evaluator 로 평가

사용자 정의 LLM 평가자를 구성하거나 Heuristic 평가자를 구성할 수 있습니다.

In [ ]:
# 설치
# !pip install -U langsmith langchain-teddynote

In [ ]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

In [ ]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH16-Evaluations")

## RAG 성능 테스트를 위한 함수 정의

테스트에 활용할 RAG 시스템을 생성하겠습니다.

In [ ]:
from myrag import PDFRAG
from langchain_openai import ChatOpenAI

# PDFRAG 객체 생성
rag = PDFRAG(
    "data/SPRI_AI_Brief_2023년12월호_F.pdf",
    ChatOpenAI(model="gpt-4o-mini", temperature=0),
)

# 검색기(retriever) 생성
retriever = rag.create_retriever()

# 체인(chain) 생성
chain = rag.create_chain(retriever)

# 질문에 대한 답변 생성
chain.invoke("삼성전자가 자체 개발한 생성형 AI의 이름은 무엇인가요?")

`ask_question` 이라는 이름으로 함수를 생성합니다. 입력으로는 `inputs` 라는 딕셔너리를 받고, 출력으로는 `answer` 라는 딕셔너리를 반환합니다.

In [ ]:
# 질문에 대한 답변하는 함수를 생성
def ask_question(inputs: dict):
    return {"answer": chain.invoke(inputs["question"])}

## 사용자 정의 Evaluator 구성

아래의 사용자 정의 함수의 입력 매개변수와 반환 값 형식을 지켜서 생성할 수 있습니다.

**사용자 정의 함수**

- 입력으로는 `Run` 과 `Example` 을 받고 출력으로는 `dict` 를 반환합니다.
- 반환 값은 `{"key": "score_name", "score": score}` 형식으로 구성됩니다.

아래는 간단한 예시 함수를 정의하였습니다. 답변에 상관없이 1~10 사이의 랜덤 점수를 반환합니다.

In [ ]:
from langsmith.schemas import Run, Example
import random


def random_score_evaluator(run: Run, example: Example) -> dict:
    # 랜덤 점수 반환
    score = random.randint(1, 10)
    return {"key": "random_score", "score": score}

In [ ]:
from langsmith.evaluation import evaluate

# 데이터셋 이름 설정
dataset_name = "RAG_EVAL_DATASET"

# 실행
experiment_results = evaluate(
    ask_question,
    data=dataset_name,
    evaluators=[random_score_evaluator],
    experiment_prefix="CUSTOM-EVAL",
    # 실험 메타데이터 지정
    metadata={
        "variant": "랜덤 점수 평가자",
    },
)

![](./assets/output-07.png)

## Custom LLM-as-Judge 

이번에는 LLM Chain 을 만들어서 평가자로 활용하겠습니다.

먼저, `context`, `answer`, `question` 을 반환하는 함수를 정의합니다.

In [ ]:
# Context 를 반환하는 RAG 결과 반환 함수
def context_answer_rag_answer(inputs: dict):
    context = retriever.invoke(inputs["question"])
    return {
        "context": "\n".join([doc.page_content for doc in context]),
        "answer": chain.invoke(inputs["question"]),
        "question": inputs["question"],
    }

다음으로는 사용자 정의 LLM 평가자를 생성합니다.

이때 평가 프롬프트는 자유롭게 조절 가능합니다.

In [ ]:
from langchain import hub

# 평가자 Prompt 가져오기
llm_evaluator_prompt = hub.pull("teddynote/context-answer-evaluator")
llm_evaluator_prompt.pretty_print()

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# 평가자 생성
custom_llm_evaluator = (
    llm_evaluator_prompt
    | ChatOpenAI(temperature=0.0, model="gpt-4o-mini")
    | StrOutputParser()
)

이전에 생성한 `context_answer_rag_answer` 함수를 사용하여 생성한 답변, 문맥을 `custom_llm_evaluator` 에 입력하여 평가를 진행합니다.

In [ ]:
# 답변을 생성합니다.
output = context_answer_rag_answer(
    {"question": "삼성전자가 자체 개발한 생성형 AI의 이름은 무엇인가요?"}
)

# 점수 평가 실행
custom_llm_evaluator.invoke(output)

`custom_evaluator` 함수를 정의합니다.

- `run.outputs`: RAG 체인이 생성한 answer, context, question 을 가져옵니다.
- `example.outputs`: 데이터셋의 정답 답변을 가져옵니다.

In [ ]:
from langsmith.schemas import Run, Example


def custom_evaluator(run: Run, example: Example) -> dict:
    # LLM 생성 답변, 정답 답변 가져오기
    llm_answer = run.outputs.get("answer", "")
    context = run.outputs.get("context", "")
    question = example.outputs.get("question", "")

    # 랜덤 점수 반환
    score = custom_llm_evaluator.invoke(
        {"question": question, "answer": llm_answer, "context": context}
    )
    return {"key": "custom_score", "score": float(score)}

평가를 진행합니다.

In [ ]:
from langsmith.evaluation import evaluate

# 데이터셋 이름 설정
dataset_name = "RAG_EVAL_DATASET"

# 실행
experiment_results = evaluate(
    context_answer_rag_answer,
    data=dataset_name,
    evaluators=[custom_evaluator],
    experiment_prefix="CUSTOM-LLM-EVAL",
    # 실험 메타데이터 지정
    metadata={
        "variant": "Custom LLM Evaluator 활용한 평가",
    },
)

![](./assets/output-08.png)